# Launch your digital twin: Cloud harvester
This is a KWH notebook which can be used to build and launch the digital twin whose configurtion file was created with the notebook 08a. The launched digital twin will automatically connect to S³I and its model will be copied into the S³I Repository. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I and modelling language library.

In [ ]:
import s3i
import time 
import os 
from ml.tools import load_config
from ml.dt_factory import create_dt_ref
from ml.app_logger import setup_logger
from tools import print_with_timestamp, yes, no, hmi_id, hmi_secret

## Load the configuration file 

In [ ]:
dt_name = input("[S³I]: Please enter the name of your digital twin (my_cloud_harvester): ")
config_file_name = "{}.json".format(dt_name)
root_path = os.path.abspath("")
dt_model = load_config(config_file_name, root=root_path)


## Set up a logger

In [ ]:
setup_logger(dt_model["attributes"].get("name", None))

## Build the digital twin 
The construction of the digital twin is based on the configuration file created previously. Features, roles and another relevant entries, which are configured in the configuration file, will be instantiated with the resptive object of the fml40 library. 

In [ ]:
dt_secret = input("[S³I]: Please enter the secret of your digital twin: ")
dt = create_dt_ref(model=dt_model, grant_type="client_credentials", 
                   secret=dt_secret,
                   is_broker=False, is_repo=True)

We offer hereby a pre-defined simulate function, which simulates the rpm changing of harvester. 

In [ ]:
def simulate_rpm():
    """
    This function simulates how the rpm value of the engine changes between 2000 to 2500 U/min. 
    """
    tank = "up"

    while True:
        if tank == "down":
            __new_rpm = dt.features["ml40::RotationalSpeed"].rpm - 10
            if __new_rpm < 2000:
                tank = "up"

        elif tank == "up":
            __new_rpm = dt.features["ml40::RotationalSpeed"].rpm + 10
            if __new_rpm > 2500:
                tank = "down"
        dt.features["ml40::RotationalSpeed"].rpm = __new_rpm
        time.sleep(1)

Insert the class and function into the instance of the digital twin. 

In [ ]:
dt.add_user_def(func=simulate_rpm)

## Launch the digital twin 
The digital twin is then started and connect to S³I automatically. It will firstly authenticate itself at the S³I IdentityProvider. The digital twin has a thing entry in the S³I Directory and a cloud copy stored in the S³I Repository, which is synchronized with the simulated rpm value.

In [ ]:
dt.run_forever()
while True:
    i = input("[S³I]: End the digital twin? (j/n)")
    if i in yes:
        break
    elif i in no:
        continue